This notebook reproduces all of the figures that involve solving the shallow water or homogenized SW equations numerically.

The pseudospectral simulations run quickly so we just execute them here in the notebook.  The finite volume simulations take longer, so we simply load them from files.  To re-create those files from scratch, install Clawpack and then run the following three commands from a terminal in the same directory as this notebook:

```
>>> python SW_periodic_bathy.py mx=20000 solver_type=sharpclaw xupper=400 tfinal=180 riemann_solver=fwave outdir=./_output_flat_20k bathy=constant b_amp=0.7

>>> python SW_periodic_bathy.py mx=40000 solver_type=sharpclaw xupper=400 tfinal=180 riemann_solver=fwave outdir=./_output_sharpclaw_mx40k/ bathy=pwc b_amp=0.7

>>> python SW_periodic_bathy.py mx=320000 solver_type=classic xupper=500 tfinal=200 riemann_solver=fwave outdir=./_output_sinusoidal_classic_b08_mx320k/ bathy=sinusoidal b_amp=0.8

```

In [ ]:
import matplotlib.pyplot as plt
from clawpack import pyclaw
from homog_pseudospectral import *
font = {'size'   : 15}
matplotlib.rc('font', **font)
lw = 1.5

# Figure 1
The next cell reproduces the example from the introduction.

In [ ]:
tmax = 180
fig, ax = plt.subplots(2,2,figsize=(12,8),sharey=True)
plt.subplots_adjust(wspace=0.02)
ytext = 0.0225
props = dict(facecolor='lightgrey', alpha=0.2)

t_int = tmax/100

def add_plot(frame,axis,xmin,xmax):
    i = frame
    axis = axis
    axis.set_ylim(-0.002,0.025)
    flat_bottom     = pyclaw.Solution(i,file_format='petsc',file_prefix='claw',path='./_output_flat_20k/')
    variable_bottom = pyclaw.Solution(i,file_format='petsc',file_prefix='claw',path='./_output_sharpclaw_mx40k/')
    x_flat = flat_bottom.grid.x.centers
    x_vari = variable_bottom.grid.x.centers
    h_flat = flat_bottom.q[0,:]
    h_vari = variable_bottom.q[0,:]
    b_flat = flat_bottom.aux[0,:]
    b_vari = variable_bottom.aux[0,:]
    eta_flat = h_flat + b_flat
    eta_vari = h_vari + b_vari
    axis.plot(x_vari,eta_vari,'-k',lw=lw)
    axis.plot(x_flat,eta_flat,'--C0',lw=lw,alpha=1)
    axis.set_xlim(xmin,xmax)
    axis.set_xticks(range(int(xmin)+5, int(xmax)-4, 10))
    axis.set_yticks([0, 0.025])  # set only 3 y-axis tick labels
    axis.text(xmin+(xmax-xmin)/3,ytext,'$t=%3.1f$' % (i*t_int),bbox=props)

add_plot(2,axis=ax[0,0],xmin=0,xmax=30)
add_plot(14,axis=ax[0,1],xmin=40,xmax=70)
add_plot(46,axis=ax[1,0],xmin=160,xmax=190)
add_plot(100,axis=ax[1,1],xmin=370,xmax=400)

fig.text(0.5, 0.04, 'x', ha='center', va='center')
fig.text(0.06, 0.5, 'Surface Height ($\eta$)', ha='center', va='center', rotation='vertical')
fig.legend(['Variable bottom','Flat bottom'],loc='upper center', fancybox=True, shadow=True);
plt.savefig('intro_example.pdf')

# Figure 5
Next we reproduce Figure 5, which is a comparison of homogenized and direct solutions for waves over piecewise-constant bathymetry.

In [ ]:
bathy = 'pwc' # Piecewise-constant bathymetry
b_amp = 0.7   # Difference in height between deep and shallow sections
m = 4096      # Number of points in space for simulation
dtfactor = 0.5 # Time step size multiplier
L = 800       # Domain goes from -L/2 to L/2
width = 3.    # Width of initial perturbation
h_amp = 0.025 # Height of initial perturbation

In [ ]:
# 3rd-order approximate series
order4 = False
order5 = False
x3, xi3, momentum3, eta3, _ = solve_SWH(bathy=bathy,h_amp=h_amp,tmax=180,m=m,width=width,L=L,IC='pulse',
                                        b_amp=b_amp,order4=order4,order5=order5,dtfac=dtfactor,
                                        make_anim=False)

In [ ]:
# 5th-order approximate series
order4 = True
order5 = True
x5, xi5, momentum5, eta5, _ = solve_SWH(bathy=bathy,h_amp=h_amp,tmax=180,m=m,width=width,L=L,IC='pulse',
                                        b_amp=b_amp,order4=order4,order5=order5,dtfac=dtfactor,
                                        make_anim=False)

In [ ]:
def load_frame(i,path):
    # Load a highly-resolved Clawpack solution
    sw_solution = pyclaw.Solution(i,file_format='petsc',file_prefix='claw',path=path)
    x_sw = sw_solution.grid.x.centers
    h_sw = sw_solution.q[0,:]
    b_sw = sw_solution.aux[0,:]
    eta_sw = h_sw + b_sw
    return x_sw, eta_sw

In [ ]:
tmax = 180
fig, ax = plt.subplots(2,2,figsize=(12,8),sharey=True)
plt.subplots_adjust(wspace=0.02)
ytext = 0.0225
props = dict(facecolor='lightgrey', alpha=0.2)
path='./_output_sharpclaw_mx40k/'
t_int = tmax/(len(eta3)-1)

def add_plot(frame,axis,xmin,xmax):
    i = frame
    axis = axis
    x_ps3, e_ps3 = fine_resolution(eta3[i],20000,x3,xi3)
    axis.plot(x_ps3,e_ps3,'-.C1',lw=lw)
    x_ps5, e_ps5 = fine_resolution(eta5[i],20000,x5,xi5)
    axis.plot(x_ps5,e_ps5,'--C0',lw=lw)
    axis.set_ylim(-0.002,0.025)
    x_sw, eta_sw = load_frame(i,path)
    axis.plot(x_sw,eta_sw,'-k',lw=lw,alpha=1.,label='Shallow Water')
    axis.set_xlim(xmin,xmax)
    axis.set_xticks(range(int(xmin)+5, int(xmax)-4, 10))
    axis.set_yticks([0, 0.025])  # set only 3 y-axis tick labels
    axis.text(xmin+(xmax-xmin)/3,ytext,'$t=%3.1f$' % (i*t_int),bbox=props)

add_plot(2,axis=ax[0,0],xmin=0,xmax=30)
add_plot(14,axis=ax[0,1],xmin=40,xmax=70)
add_plot(46,axis=ax[1,0],xmin=160,xmax=190)
add_plot(100,axis=ax[1,1],xmin=370,xmax=400)

fig.text(0.5, 0.04, 'x', ha='center', va='center')
fig.text(0.06, 0.5, 'Surface Height ($\eta$)', ha='center', va='center', rotation='vertical')
fig.legend(['$O(\delta^3)$ Homogenized','$O(\delta^5)$ Homogenized','Shallow water'],loc='upper center', fancybox=True, shadow=True);
plt.savefig('comparison_pwc.pdf')

# Figure 6
Figure 6 compares the direct solution with a homogenized solution that includes both the slow- and fast-scale effects.

In [ ]:
# 5th-order approximate plus 7th-order linear terms
order4 = True
order5 = True
order7 = True
x7, xi7, momentum7, eta7, _ = solve_SWH(bathy=bathy,h_amp=h_amp,tmax=180,m=m,width=width,L=L,IC='pulse',
                                        b_amp=b_amp,order4=order4,order5=order5,order7=order7,dtfac=dtfactor,
                                        make_anim=False)

In [ ]:
i = 14
N_plot_points = 20000
xfine7, etafine7 = fine_resolution(eta7[i],N_plot_points,x7,xi7)
xfine7, momfine7 = fine_resolution(momentum7[i],N_plot_points,x7,xi7)
eta_fast = eta_variation(xfine7,etafine7,momfine7,bathy='tanh',b_amp=b_amp)

In [ ]:
# Load a highly-resolved Clawpack solution
sw_solution = pyclaw.Solution(i,file_format='petsc',file_prefix='claw',path='./_output_sharpclaw_mx40k/')
x_sw = sw_solution.grid.x.centers
h_sw = sw_solution.q[0,:]
b_sw = sw_solution.aux[0,:]
eta_sw = h_sw + b_sw

In [ ]:
fs = 20
plt.figure(figsize=(12,6))
plt.plot(x_sw,eta_sw, '-k', lw=lw, label='Shallow water')
plt.plot(xfine7, etafine7, '--C0', lw=1.5, label='$\overline{\eta}(x,t)$')
plt.plot(xfine7, etafine7+eta_fast, '--C3', label='$\eta(x,y,t)$')
plt.xlim(45,62)
plt.legend(fontsize=fs);
plt.ylabel('$\eta$',fontsize=fs)
plt.xlabel('x',fontsize=fs)
plt.savefig('fast_variation.pdf')

# Figure 7
Figure 7 is a comparison of homogenized and direct solutions for waves over sinusoidal bathymetry.

In [ ]:
b_amp = 0.8
bathy = 'sinusoidal'
width = 3.
h_amp = 0.025
tmax = 200
m = 4096
L = 1000

In [ ]:
# 3rd-order approximate series
order4 = False
order5 = False
x3, xi3, momentum3, eta3, _ = solve_SWH(bathy=bathy,h_amp=h_amp,tmax=tmax,m=m,width=width,L=L,IC='pulse',
                                        b_amp=b_amp,order4=order4,order5=order5,dtfac=dtfactor,
                                        make_anim=False)

In [ ]:
# 5th-order approximate series
order4 = True
order5 = True
x5, xi5, momentum5, eta5, _ = solve_SWH(bathy=bathy,h_amp=h_amp,tmax=tmax,m=m,width=width,L=L,IC='pulse',
                                        b_amp=b_amp,order4=order4,order5=order5,dtfac=dtfactor,
                                        make_anim=False)

In [ ]:
fig, ax = plt.subplots(2,2,figsize=(12,8),sharey=True)
plt.subplots_adjust(wspace=0.02)
ytext = 0.0225
props = dict(facecolor='lightgrey', alpha=0.2)
path='./_output_sinusoidal_classic_b08_mx320k/'
t_int = tmax/(len(eta3)-1)

def add_plot(frame,axis,xmin,xmax):
    i = frame
    axis = axis
    x_ps3, e_ps3 = fine_resolution(eta3[i],20000,x3,xi3)
    axis.plot(x_ps3,e_ps3,'-.C1',lw=lw)
    x_ps5, e_ps5 = fine_resolution(eta5[i],20000,x5,xi5)
    axis.plot(x_ps5,e_ps5,'--C0',lw=lw)
    axis.set_ylim(-0.002,0.025)
    x_sw, eta_sw = load_frame(i,path)
    axis.plot(x_sw,eta_sw,'-k',lw=lw,alpha=1.,label='Shallow Water')
    axis.set_xlim(xmin,xmax)
    axis.set_xticks(range(int(xmin)+5, int(xmax)-4, 10))
    axis.set_yticks([0, 0.025])  # set only 3 y-axis tick labels
    axis.text(xmin+(xmax-xmin)/3,ytext,'$t=%3.1f$' % (i*t_int),bbox=props)

add_plot(2,axis=ax[0,0],xmin=0,xmax=30)
add_plot(14,axis=ax[0,1],xmin=40,xmax=70)
add_plot(46,axis=ax[1,0],xmin=180,xmax=210)
add_plot(100,axis=ax[1,1],xmin=410,xmax=440)

fig.text(0.5, 0.04, 'x', ha='center', va='center')
fig.text(0.06, 0.5, 'Surface Height ($\eta$)', ha='center', va='center', rotation='vertical')
fig.legend(['$O(\delta^3)$ Homogenized','$O(\delta^5)$ Homogenized','Shallow water'],loc='upper center', fancybox=True, shadow=True);
plt.savefig('comparison_sinusoidal.pdf')